In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [2]:
import re
import pytest
import numpy as np
import pandas as pd
from sklearn.exceptions import NotFittedError
from skforecast.preprocessing import RollingFeatures
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor
from lightgbm import LGBMRegressor

In [29]:
# Create and fit forecaster
# ==============================================================================
rolling = RollingFeatures(stats= ['mean', 'median', 'sum'], window_sizes=[5, 4, 5])
forecaster = ForecasterAutoregDirect(
                 regressor       = LinearRegression(),
                 steps           = 2,
                 lags            = None,
                 window_features= rolling,
                 differentiation = 1
             )

y = pd.Series(np.arange(10), name='y')
exog = pd.Series(np.arange(100, 110), name='exog')
exog_predict = pd.Series(np.arange(110, 112), name='exog')
exog_predict.index = pd.RangeIndex(start=10, stop=12, step=1)
forecaster.fit(y=y, exog=exog)

In [30]:
forecaster._create_train_X_y(y=y, exog=exog)

(   roll_mean_5  roll_median_4  roll_sum_5  exog_step_1  exog_step_2
 7          1.0            1.0         5.0        106.0        107.0
 8          1.0            1.0         5.0        107.0        108.0
 9          1.0            1.0         5.0        108.0        109.0,
 {1: 6    1.0
  7    1.0
  8    1.0
  Name: y_step_1, dtype: float64,
  2: 7    1.0
  8    1.0
  9    1.0
  Name: y_step_2, dtype: float64},
 ['exog'],
 ['roll_mean_5', 'roll_median_4', 'roll_sum_5'],
 ['exog'],
 ['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog_step_1', 'exog_step_2'],
 {'exog': dtype('int64')})

In [93]:
forecaster.create_predict_X(exog=exog_predict)

['roll_mean_5', 'roll_median_4', 'roll_sum_5', 'exog']


,roll_mean_5,roll_median_4,roll_sum_5,exog
10,7.0,7.5,35.0,110.0
11,7.0,7.5,35.0,111.0


In [95]:
forecaster.predict(exog=exog_predict)

10    10.0
11    11.0
Name: pred, dtype: float64

In [35]:
rolling.transform_batch(y)

,roll_mean_4,roll_median_4,roll_sum_5
5,2.5,2.5,10.0
6,3.5,3.5,15.0
7,4.5,4.5,20.0
8,5.5,5.5,25.0
9,6.5,6.5,30.0


In [96]:
y = pd.Series(np.arange(5), name='y')
forecaster = ForecasterAutoregDirect(LinearRegression(), lags=3, steps=2)

forecaster.create_train_X_y(y=y)

(   lag_1  lag_2  lag_3
 4    2.0    1.0    0.0,
 {1: 3    3.0
  Name: y_step_1, dtype: float64,
  2: 4    4.0
  Name: y_step_2, dtype: float64})

In [6]:
from skforecast.preprocessing import RollingFeatures
from skforecast.preprocessing import TimeSeriesDifferentiator
from skforecast.ForecasterAutoreg import ForecasterAutoreg

# Fixtures
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import y
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import exog_predict
from skforecast.ForecasterAutoreg.tests.fixtures_ForecasterAutoreg import data  # to test results when using differentiation


In [19]:
y_datetime = y.copy()
y_datetime.index = pd.date_range(start='2001-01-01', periods=len(y), freq='D')
exog_datetime = exog.copy()
exog_datetime.index = pd.date_range(start='2001-01-01', periods=len(exog), freq='D')
exog_predict_datetime = exog_predict.copy()
exog_predict_datetime.index = pd.date_range(start='2001-02-20', periods=len(exog_predict), freq='D')
rolling = RollingFeatures(stats=['mean', 'sum'], window_sizes=3)
forecaster = ForecasterAutoreg(
    LGBMRegressor(verbose=-1, random_state=123), lags=3, window_features=rolling,
    differentiation=1
)
forecaster.fit(y=y_datetime, exog=exog_datetime)

In [ ]:

results = forecaster.predict_bootstrapping(
    steps=4, n_boot=10, exog=exog_predict_datetime, use_in_sample_residuals=True
)
results

In [20]:
y_datetime = pd.Series(
    np.arange(15), index=pd.date_range('2000-01-01', periods=15, freq='D'),
    name='y', dtype=float
)
exog_datetime = pd.Series(
    np.arange(100, 115), index=pd.date_range('2000-01-01', periods=15, freq='D'),
    name='exog', dtype=float
)
rolling = RollingFeatures(
    stats=['mean', 'median', 'sum'], window_sizes=[5, 5, 6]
)

forecaster = ForecasterAutoregDirect(
    LinearRegression(), steps=2, lags=5, window_features=rolling
)
results = forecaster._create_train_X_y(y=y_datetime, exog=exog_datetime)
results

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
0 -1
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13.]
DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10', '2000-01-11', '2000-01-12',
               '2000-01-13', '2000-01-14'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2000-01-08', '2000-01-09', '2000-01-10', '2000-01-11',
               '2000-01-12', '2000-01-13', '2000-01-14', '2000-01-15'],
              dtype='datetime64[ns]', freq='D')


(            lag_1  lag_2  lag_3  lag_4  lag_5  roll_mean_5  roll_median_5  \
 2000-01-08    5.0    4.0    3.0    2.0    1.0          3.0            3.0   
 2000-01-09    6.0    5.0    4.0    3.0    2.0          4.0            4.0   
 2000-01-10    7.0    6.0    5.0    4.0    3.0          5.0            5.0   
 2000-01-11    8.0    7.0    6.0    5.0    4.0          6.0            6.0   
 2000-01-12    9.0    8.0    7.0    6.0    5.0          7.0            7.0   
 2000-01-13   10.0    9.0    8.0    7.0    6.0          8.0            8.0   
 2000-01-14   11.0   10.0    9.0    8.0    7.0          9.0            9.0   
 2000-01-15   12.0   11.0   10.0    9.0    8.0         10.0           10.0   
 
             roll_sum_6  exog_step_1  exog_step_2  
 2000-01-08        15.0        106.0        107.0  
 2000-01-09        21.0        107.0        108.0  
 2000-01-10        27.0        108.0        109.0  
 2000-01-11        33.0        109.0        110.0  
 2000-01-12        39.0        110.0

In [24]:
y_datetime = pd.Series(
    [25.3, 29.1, 27.5, 24.3, 2.1, 46.5, 31.3, 87.1, 133.5, 4.3],
    index=pd.date_range('2000-01-01', periods=10, freq='D'),
    name='y', dtype=float
)
exog = pd.DataFrame({
            'col_1': [7.5, 24.4, 60.3, 57.3, 50.7, 41.4, 87.2, 47.4, 14.6, 73.5],
            'col_2': ['a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'b']},
            index = pd.date_range('2000-01-01', periods=10, freq='D')
        )

transformer_y = StandardScaler()
transformer_exog = ColumnTransformer(
                        [('scale', StandardScaler(), ['col_1']),
                            ('onehot', OneHotEncoder(), ['col_2'])],
                        remainder = 'passthrough',
                        verbose_feature_names_out = False
                    )
rolling = RollingFeatures(
    stats=['ratio_min_max', 'median'], window_sizes=4
)

forecaster = ForecasterAutoregDirect(
                    LinearRegression(), 
                    steps            = 2,
                    lags             = [1, 5], 
                    window_features  = rolling,
                    transformer_y    = transformer_y,
                    transformer_exog = transformer_exog,
                    differentiation  = 1
                )
results = forecaster._create_train_X_y(y=y_datetime, exog=exog)
results

[        nan  0.09974117 -0.04199628 -0.08399257 -0.58269844  1.16539688
 -0.3989647   1.46462041  1.21789224 -3.39119993]
1 -1
[ 0.09974117 -0.04199628 -0.08399257 -0.58269844  1.16539688 -0.3989647
  1.46462041  1.21789224]
DatetimeIndex(['2000-01-02', '2000-01-03', '2000-01-04', '2000-01-05',
               '2000-01-06', '2000-01-07', '2000-01-08', '2000-01-09'],
              dtype='datetime64[ns]', freq='D')
DatetimeIndex(['2000-01-08', '2000-01-09', '2000-01-10'], dtype='datetime64[ns]', freq='D')


(               lag_1     lag_5  roll_ratio_min_max_4  roll_median_4  \
 2000-01-08  1.165397  0.099741             -0.500000      -0.062994   
 2000-01-09 -0.398965 -0.041996             -0.500000      -0.241479   
 2000-01-10  1.464620 -0.083993             -0.397849       0.383216   
 
             col_1_step_1  col_2_a_step_1  col_2_b_step_1  col_1_step_2  \
 2000-01-08      1.698160             0.0             1.0      0.040403   
 2000-01-09      0.040403             0.0             1.0     -1.325790   
 2000-01-10     -1.325790             0.0             1.0      1.127525   
 
             col_2_a_step_2  col_2_b_step_2  
 2000-01-08             0.0             1.0  
 2000-01-09             0.0             1.0  
 2000-01-10             0.0             1.0  ,
 {1: 2000-01-07   -0.398965
  2000-01-08    1.464620
  2000-01-09    1.217892
  Freq: D, Name: y_step_1, dtype: float64,
  2: 2000-01-08    1.464620
  2000-01-09    1.217892
  2000-01-10   -3.391200
  Freq: D, Name: y_step_

In [30]:
results[1][2].to_numpy()

array([ 1.46462041,  1.21789224, -3.39119993])